Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
from scipy.fft import fft, ifft
import pickle

from kpz import *
from inverse import *

In [ ]:
n = 100
t = np.array([10, 30])

Load Daryn's data

In [ ]:
# load real data
filename = 'spread.pickle'
with open(filename, 'rb') as f:
    spread = pickle.load(f)

Extract the firelines of interest

In [ ]:
firstFireline = spread[t[0]]
firstx = firstFireline[0]
firsth = firstFireline[1]
plt.plot(firstx, firsth, label='first fireline')

lastFireline = spread[t[1]]
lastx = lastFireline[0]
lasth = lastFireline[1]
plt.plot(lastx, lasth, label='last fireline')
plt.legend()

Truncate the firelines to only include the portion of the fireline that is spreading. Ie not the sides

Currently the bounds are specified manually

In [ ]:
x0 = 100
xn = 250
x_bounds = np.array([x0, xn])
firstx = truncate(firstx, x_bounds)
firsth = truncate(firsth, x_bounds)
lastx = truncate(lastx, x_bounds)
lasth = truncate(lasth, x_bounds)
plt.plot(firstx, firsth, label='first fireline')
plt.plot(lastx, lasth, label='last fireline')
plt.legend()

Perform the inverse method

In [ ]:
h, x, ha_result, y_bounds = inverse(np.array([firsth, lasth]), t, n)
ha_result

Show the fitted parameters

In [ ]:
a_opt = ha_result.x
final = solveKPZ(h[0], t, a_opt)
plt.clf()
plt.plot(x, h[0], 'k-', label='first fireline')
plt.plot(x, h[1], 'b-', label='final fireline')
plt.plot(x, final.y[:, -1], 'r-', label='final reconstructed')
plt.legend()

Verify the solution by comparing to a fireline it hasn't seen

In [ ]:
# verify the inverse method by predicting a later fireline
t[1] = 50
verifyFireline = spread[t[1]]
verifyh = verifyFireline[1]
verifyh = truncate(verifyh, (x0, xn))
verifyh_smooth = smooth(verifyh, n)
verifyh_smooth = normalize(verifyh_smooth, y_bounds)
verify = solveKPZ(h[0], t, a_opt)
plt.clf()
plt.plot(x, h[0], 'k-', label='first fireline')
plt.plot(x, h[1], 'b-', label='final fireline')
plt.plot(x, verifyh_smooth, 'g-', label='verification fireline')
plt.plot(x, verify.y[:, -1], 'r-', label='verification reconstructed')
plt.legend()

Calculate distribution of the deviation/roughness

In [ ]:
err = verifyh_smooth - verify.y[:, -1]
err.shape
plt.clf()
_ = plt.hist(err, bins=100)

margin of error for c v lamb

find distributions for c v lamb

and find power law (in paper about burning paper)

basically, investigating what's wrong with the KPZ equation and what it's failing to capture. if c v lamb don't capture much if any useful information about the spread, then the model isn't so good and i'll need a new equation

markov chain mc model for c v lamb